<span style="float: left;padding: 1.3em">![logo](https://github.com/gw-odw/odw/blob/main/Tutorials/logo.png?raw=1)</span>

# Gravitational Wave Open Data Workshop

## Tutorial 2.3: PyCBC Tutorial, Signal Consistency and Significance

We will be using the [PyCBC](https://pycbc.org) library, which is used to study gravitational-wave data, find astrophysical sources due to compact binary mergers, and study their parameters. These are some of the same tools that the LIGO and Virgo collaborations use to find gravitational waves in LIGO/Virgo data

In this tutorial we will walk through finding a peak in a noisy timeseries and estimating its significance given a simplified search. Some assumptions will be noted along the way. We will also make use of one of the standard signal consistency tests to help remove some non-Gaussian transient noise from the background.

View this tutorial on [Google Colaboratory](https://colab.research.google.com/github/gw-odw/odw/blob/main/Tutorials/Day_2/Tuto_2.3_Signal_consistency_and_significance.ipynb) or launch [mybinder](https://mybinder.org/v2/gh/gw-odw/odw/HEAD).

See [additional examples](https://pycbc.org/pycbc/latest/html/#library-examples-and-interactive-tutorials) and [documentation](https://pycbc.org/pycbc/latest/html/index.html).

In [2]:
# Those 2 lines are just to avoid some harmless warnings when importing packages
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")

## Installation (execute only if running on a cloud platform, like Google Colab, or if you haven't done the installation already!)

> ⚠️ **Warning**: restart the runtime after running the cell below.
>
> To do so, click "Runtime" in the menu and choose "Restart and run all".

In [13]:
# -- Use the following for Google Colab
! pip install -q 'lalsuite==7.25' 'PyCBC==2.4.1'
! pip install numpy==1.25.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 74.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blosc2 3.3.2 requires numpy>=1.26, but you have numpy 1.25.2 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.25.2 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.25.2 which is incompatible.


### Significance of Virgo SNR peak of GW170814 ###

We will estimate the significance of signal-to-noise peak observed in the Virgo instrument coincident with the large peaks observed in the LIGO-Hanford and LIGO-Livingston observatories.

For this purpose we will consider a gravitational wave signal, whose existence has been confirmed based on the signals from LIGO detectors alone. This was in fact the case for the matched-filtering based analyses of GW170814, as they did not incorporate any information from the Virgo data.

We ask ourselves the following question: *What is the probability that noise can produce a peak as large or larger than the largest peak observed in the Virgo data, and consistent with the lightspeed travel time between all three observatories?*

This is a form of null hypothesis testing, where we compute a [p-value](https://en.wikipedia.org/wiki/P-value) to answer the question above.

<!-- For the purpose of this notebook, we have added a few additional simplifying assumptions, and those will be stated as we go along. -->

#### Read and Precondition Gravitational Strain Data ####

In this section, we will read in a short segment of data around GW170814, and do some basic preconditioning as demonstrated in previous tutorials. We will also calculate the power spectrum of the data.

Notably, here we assume that the power spectrum estimated from the data is constant over the short stretch of time, and isn't biased by our choice to center the estimate (very roughly) on the event time. We *do not* assume the data to be stationary, Gaussian, or free from non-astrophysical transient artefacts.

In [ ]:
# The first import of matplotlib can take some time (especially on cloud platforms). This is normal.
import matplotlib.pyplot as plt

from pycbc.filter import resample_to_delta_t, highpass
from pycbc.catalog import Merger
from pycbc.psd import interpolate, inverse_spectrum_truncation

m = Merger("GW170814")

ifos = ['H1', 'L1', 'V1']
data = {}
psd = {}

plt.figure(figsize=[10, 5])

for ifo in ifos:
    # Read in and precondition the data
    ts = m.strain(ifo).highpass_fir(15, 512)
    data[ifo] = resample_to_delta_t(ts, 1.0/2048).crop(2, 2)

    # Estimate the power spectral density of the data
    # This chooses to use 2s samples in the PSD estimate.
    # One should note that the tradeoff in segment length is that
    # resolving narrow lines becomes more difficult.
    p = data[ifo].psd(2)
    p = interpolate(p, data[ifo].delta_f)
    p = inverse_spectrum_truncation(p, int(2 * data[ifo].sample_rate), low_frequency_cutoff=15.0)
    psd[ifo] = p

    plt.plot(psd[ifo].sample_frequencies, psd[ifo], label=ifo)

plt.yscale('log')
plt.xscale('log')
plt.ylim(1e-47, 1e-41)
plt.xlim(20, 1024)
plt.ylabel('$Strain^2 / Hz$')
plt.xlabel('Frequency (Hz)')
plt.grid()
plt.legend()
plt.show()

/usr/local/lib/python3.11/dist-packages/pycbc/types/array.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal as _lal


#### Generate our template waveform and calculate the Signal-to-noise time series ####

To calculate the signal-to-noise time series, we need to generate an estimate of the signal to use as a template. For this purpose we will assume the source black holes are non spinning, have equal mass, and they agree with the total mass estimate for the system as a whole. A better method would be to use the maximum likelihood estimate from an analysis of the LIGO data alone, however, this is sufficient for the purposes of this tutorial.

In [ ]:
from pycbc.waveform import get_fd_waveform
from pycbc.filter import matched_filter

# Calculate the component mass of each black hole in the detector frame
cmass = (m.median1d("mass1")+m.median1d("mass2")) / 2      # This is in the source frame
cmass *= (1 + m.median1d("redshift")) # apply redshift to get to the detector frame

# This is a frequency domain waveform generator. It has a very similar syntax to the time domain
# waveform function used in prior tutorials. This function returns both a plus and a cross
# polarization waveform, but we will just use the plus polarization in building our template
# as these are only different by a phase offset in this specific case.
hp, _ = get_fd_waveform(approximant="IMRPhenomD",
                         mass1=cmass, mass2=cmass,
                         f_lower=20.0, delta_f=data[ifo].delta_f)
hp.resize(len(psd[ifo]))

# For each observatory use this template to calculate the SNR time series
snr = {}
for ifo in ifos:
    snr[ifo] = matched_filter(hp, data[ifo], psd=psd[ifo], low_frequency_cutoff=20)
    snr[ifo] = snr[ifo].crop(5, 4)

In [ ]:
# Show a couple sizes
for w, title in [(8, 'Wide View'), (.15, 'Close to GW170814')]:
    plt.figure(figsize=[14, 4])
    for ifo in ifos:
        plt.plot(snr[ifo].sample_times, abs(snr[ifo]), label=ifo)

    plt.legend()
    plt.title(title)
    plt.grid()
    plt.xlim(m.time - w, m.time + w)
    plt.ylim(0, 15)
    plt.xlabel('Time (s)')
    plt.ylabel('Signal-to-noise (SNR)')
    plt.show()

We see in the SNR time series plots above that while there are nice peaks around GW170814 in each detector, there are also some large peaks at other times. LIGO / Virgo data does contain transient (i.e limited duration) noise artefacts that an analysis must deal with to search LIGO data with high sensitivity. One approach for dealing with this is outlined later in this tutorial.

#### How well is the data actually fitting our model? ####

One of the ways we can test how well the data actual fits the models to use a $\chi^2$-based signal consistency test. We employ a version of the test described [in this paper](https://arxiv.org/pdf/gr-qc/0405045.pdf). Schematically, we chop up our template into $p$ number of frequency bins and see how much the SNR in each individual bin contributes to the total SNR ($\rho_i$) of the supposed signal. The frequency bins are defined such that we expect that there should be a roughly equal contribution to the SNR from each. We can then calculate our statistic as the difference between the SNR in one bin, and the expected fraction of the total SNR ($\rho$).

$
\chi^2 = \sum^p_{i=0} (\rho_i - \rho / p)^2
$

This will have $2p-2$ degrees of freedom as each SNR is *complex* representing both possible orthogonal phases the signal could have contributions from. There is also a constraint due to the fact that the sum of each bin must each add up to the total SNR by definition. In this notebook we will normalize this statistic by dividing by the number of degrees of freedom, producing $\chi^2_r$.

We expect that this statistic will be high when the template does not match well with the data, and near unity when the data either is Gaussian noise, or it contains the expected signal in addition to Gaussian noise.

In [ ]:
# WARNING!! If you are having problems with this code, replace the import with
#from pycbc_chisq import power_chisq
from pycbc.vetoes import power_chisq

chisq = {}
for ifo in ifos:
    # The number of bins to use. In principle, this choice is arbitrary. In practice,
    # this is empirically tuned.
    nbins = 26
    chisq[ifo] = power_chisq(hp, data[ifo], nbins, psd[ifo], low_frequency_cutoff=20.0)
    chisq[ifo] = chisq[ifo].crop(5, 4)

    dof = nbins * 2 - 2
    chisq[ifo] /= dof

In [ ]:
plt.figure(figsize=[14, 4])

for ifo in ifos:
    plt.plot(chisq[ifo].sample_times, chisq[ifo], label=ifo)

plt.legend()
plt.grid()
plt.xlim(m.time -0.15, m.time + 0.15)
plt.ylim(0, 5)
plt.ylabel('$chi^2_r$')
plt.show()

There are some notable features in the $\chi^2_r$ time series. We see that there is a dip in the value at the time of the peak in the SNR in each observatory. We expect this as the template now aligns with the signal in the data. Also, the values climb just around this minima. This occurs because the template is starting to slide against the true signal in the data but is not perfectly aligned with it.

#### Re-weighting our SNR to help down-weight times that don't fit our signal ####

One approach we can take to improve the sensitivity of the search is to down-weight the times where the data does not appear as either Gaussian noise or Gaussian noise + our template. We can do this be combining the SNR time series and our $\chi^2_r$ time series. This is a method used to re-weight the SNR calculated by the PyCBC pipeline since initial LIGO, and has been employed in the first three Advanced LIGO observing runs. In this tutorial we will choose to use the [re-weighted SNR $\hat{\rho}$](https://pycbc.org/pycbc/latest/html/_modules/pycbc/events/ranking.html) ([described in this paper originally](https://iopscience.iop.org/article/10.1088/0264-9381/33/21/215004/pdf), with [more discussion in this work](https://iopscience.iop.org/article/10.1088/1361-6382/ab685e)) to rank our events:

$\hat{\rho} = \frac{\rho}{ \frac{1}{2}[1 + (\chi^2_r)^3]^{1/6}}$ where $\chi^2 > 1$, otherwise $\rho$

For reference on how we rank coincident (i.e. occurring in multiple detector) events in Advanced LIGO, there is a description [here](https://iopscience.iop.org/article/10.3847/1538-4357/aa8f50/pdf).

In [ ]:
from pycbc.events.ranking import newsnr

# The rho-hat term above is named "newsnr" here
nsnr = {ifo:newsnr(abs(snr[ifo]), chisq[ifo]) for ifo in ifos}

# Show a couple sizes
for w, title in [(8, 'Wide View'), (.15, 'Close to GW170814')]:
    plt.figure(figsize=[14, 4])
    for ifo in ifos:
        plt.plot(snr[ifo].sample_times, nsnr[ifo], label=ifo)

    plt.legend()
    plt.title(title)
    plt.grid()
    plt.xlim(m.time - w, m.time + w)
    plt.ylim(0, 15)
    plt.xlabel('Time (s)')
    plt.ylabel('Re-weighted Signal-to-noise')
    plt.show()

We can see above that there are still peaks around GW170814 in all detectors at roughly the same signal strength. At other times - where there were previously peaks in the time series due to transient noise - there are no longer large SNR values.

#### Calculating the background and significance ####

In this section we will determine how significant the peak in the Virgo re-weighted SNR time series is.

We will do this first by determining where one might expect a peak associated with an astrophysical source relative to the LIGO observed peaks. This is set by the constraint that an astrophysical source can only cause delays between observatories no larger than the light travel time between them. The [`pycbc.detector.Detector`](https://pycbc.org/pycbc/latest/html/pycbc.html#pycbc.detector.Detector) class provides some convenient methods to calculate the light travel time between detectors.

We will then identify the largest peak in the SNR for this window around the LIGO observed peaks. This is our "on-source" window.

Finally, to determine the significance of the peak detected in the on-source window, we will compare how likely it is for a peak as large or larger to appear in the background. Our background will be empirically measured by taking portions of the SNR time series from an "off-source" window i.e. times where it is not possible for the signal to travel from one detector to another within the light travel time.

An important criteria to avoid a biased significance estimate is that algorithm to identify the background (off-source) and foreground (on-source) peaks is the same.

In [ ]:
import numpy
from pycbc.detector import Detector

# Calculate the time of flight between the Virgo detectors and each LIGO observatory
d = Detector("V1")
tof = {}
tof['H1'] = d.light_travel_time_to_detector(Detector("H1"))
tof['L1'] = d.light_travel_time_to_detector(Detector("L1"))

# Record the time of the peak in the LIGO observatories
ptime = {}

plt.figure(figsize=[14, 4])
for ifo in ifos:

    # shade the region around each LIGO peak that could have a peak in Virgo if from
    # an astrophysical source
    if ifo != 'V1':
        ptime[ifo] = snr[ifo].sample_times[nsnr[ifo].argmax()]
        plt.axvspan(ptime[ifo] - tof[ifo], ptime[ifo] + tof[ifo], alpha=0.2, lw=10)

    plt.plot(snr[ifo].sample_times, nsnr[ifo], label=ifo)

# Calculate the span of time that a Virgo peak could in principle happen in from time of flight
# considerations.
start = ptime['H1'] - tof['H1']
end = ptime['L1'] + tof['L1']

# convert the times to indices along with how large the region is in number of samples
window_size = int((end - start) * snr['V1'].sample_rate)
sidx = int((start - snr['V1'].start_time) * snr['V1'].sample_rate)
eidx = sidx + window_size

# Calculate the "on-source" peak re-weighted (newsnr) statistic value.
onsource = nsnr['V1'][sidx:eidx].max()

plt.legend()
plt.grid()
plt.xlim(m.time - .08, m.time + .08)
plt.ylim(0, 15)
plt.xlabel('Time (s)')
plt.ylabel('Re-weighted Signal-to-noise')
plt.show()

print('Virgo Peak has a re-weighted SNR value of {}'.format(onsource))

In the plot above we see the re-weighted SNR time series. On top of that we've shaded the regions which are consistent with a Virgo signal based on the peaks in the LIGO observatories. Only in the darker region, is it possible to have a peak in the SNR that is consistent with both LIGO observatories.

In [ ]:
# Now that we've calculated the onsource peak, we should calculate the background peak values.
# We do this by chopping up the time series into chunks that are the same size as our
# on-source window and repeating the same peak finding (max) procedure - keeping the algorithm
# the same to prevent bias

# Walk through the data in chunks and calculate the peak statistic value in each.
peaks = []
i = 0
while i + window_size < len(nsnr['V1']):
    p = nsnr['V1'][i:i+window_size].max()
    peaks.append(p)
    i += window_size

    # Skip past the onsource time
    if abs(i - sidx) < window_size:
        i += window_size * 2

peaks = numpy.array(peaks)

In [ ]:
# The p-value is just the number of samples observed in the background with a
# value equal or higher than the onsource divided by the number of samples.
# We can make the mapping between statistic value and p-value using our background
# samples.
pcurve = numpy.arange(1, len(peaks)+1)[::-1] / float(len(peaks))
peaks.sort()

pvalue = (peaks > onsource).sum() / float(len(peaks))

plt.figure(figsize=[10, 7])
plt.scatter(peaks, pcurve, label='Off-source (Noise Background)', color='black')

plt.axvline(onsource, label='On-source', color='red')
plt.axhline(pvalue, color='red')

plt.legend()
plt.yscale('log')
plt.grid()
plt.ylim(1e-3, 1e0)
plt.ylabel('p-value')
plt.xlabel('Re-weighted Signal-to-noise')

plt.xlim(2, 5)
plt.show()

print("The p-value associated with the GW170814 peak is {}".format(pvalue))

In this tutorial, we find a peak in Virgo as large as the obseved one has an approximately 2% chance of occurring due to the noise alone. Since that is a relatively low probability, we may reject the null hypothesis that the observed peak is due to noise alone. Given the simplifications of this tutorial, we find a result in agreement with the [GW170814 discovery paper](https://arxiv.org/pdf/1709.09660.pdf) which reported a p-value of 0.3%. The reason for the slight discrepancy in these numbers is due to the duration over which the background is collected.

If the signal was much louder in the Virgo data, the Virgo peak would be larger than any peak in the noise background. In this case, this method of estimating the significance would only be able to set an upper bound on the p-value of the observed peak. In order to calculate the p-value of a much larger peak, we would either need to use more background data or make additional assumptions about the background distribution. If a gravitational-wave signal is extremely loud, it is challenging to calculate the precise significance of the observed peak, but we can still be confident that the signal is very significant!

## Challenge!

Use the methods demonstrated above to see if you can calculate the SNR time series and re-weighted SNR timeseries in the following data set. This data set contains one signal and two glitches. At what times do you find peaks in the SNR timeseries? Which peaks are still present in the re-weighted SNR timeseries?

Information that may be useful:

* The signal and glitches are all placed between 100 and 120 seconds into the frame file.
* You may assume mass1 = mass2 (equal mass) and that the component mass of the signal is 32.
* Each file starts at gps time 0, and ends at gps time 128
* The channel name in each file is "H1:TEST-STRAIN"

In [ ]:
# Download the challenge set files
from pycbc.frame import read_frame
import os
import urllib

def get_file(fname):
    url = "https://github.com/gw-odw/odw/raw/main/Tutorials/Day_2/{}"
    url = url.format(fname)
    urllib.request.urlretrieve(url, fname)
    print('Getting : {}'.format(url))

files = ['Data/PyCBC_T3_0.gwf']

# Download the file if needed
for fname in files:
    if not os.path.exists(fname):
        os.makedirs("Data", exist_ok=True)
        get_file(fname)


# An example of how to read the data from these files:
file_name = "Data/PyCBC_T3_0.gwf"

# LOSC bulk data typically uses the same convention for internal channels names
# Strain is typically IFO:LOSC-STRAIN, where IFO can be H1/L1/V1.
channel_name = "H1:TEST-STRAIN"

start = 0
end = start + 128

ts = read_frame(file_name, channel_name, start, end)

In [ ]:
import pylab
def challenge_matched_filter(file_name,mass):
  print("Looking at file {} with template mass {} M_sol".format(file_name,mass))
  channel_name = "H1:TEST-STRAIN"
  start = 0
  end = start + 128
  ts = read_frame(file_name, channel_name, start, end)
  ts = highpass(ts, 15.0)
  strain_ts = resample_to_delta_t(ts, 1.0/2048)
  conditioned_ts = strain_ts.crop(2, 2)
  psd_ts = conditioned_ts.psd(4)
  psd_ts = interpolate(psd_ts, conditioned_ts.delta_f)
  psd_ts = inverse_spectrum_truncation(psd_ts, int(4 * conditioned_ts.sample_rate),
  low_frequency_cutoff=15)
  hp_x, _ = get_fd_waveform(approximant="IMRPhenomD",
     mass1=mass, mass2=mass,
     f_lower=20.0, delta_f=conditioned_ts.delta_f)
  hp_x.resize(len(psd_ts))

  # For each observatory use this template to calculate the SNR time series
  snr_x = matched_filter(hp_x, conditioned_ts, psd=psd_ts, low_frequency_cutoff=20).crop(5, 4)

  pylab.figure(figsize=[14, 4])
  pylab.plot(snr_x.sample_times, abs(snr_x), label='H1')
  pylab.title('SNR Time Series')
  pylab.grid()
  pylab.xlim(100,120)
  pylab.ylim(0, 15)
  pylab.xlabel('Time (s)')
  pylab.ylabel('Signal-to-noise (SNR)')
  pylab.show()

  chisq = {}
  nbins = 26
  chisq_x = power_chisq(hp_x, conditioned_ts, nbins, psd_ts, low_frequency_cutoff=20.0)
  chisq_x = chisq_x.crop(5, 4)

  dof_x = nbins * 2 - 2
  chisq_x /= dof_x


  # The rho-hat term above is named "newsnr" here
  nsnr_x = newsnr(abs(snr_x), chisq_x)

  # Plot the new SNR timeseries
  pylab.figure(figsize=[14, 4])
  pylab.plot(snr_x.sample_times, nsnr_x, label='H1')
  pylab.title('NewSNR Timeseries')
  pylab.grid()
  pylab.xlim(100,120)
  pylab.ylim(0, 15)
  pylab.xlabel('Time (s)')
  pylab.ylabel('Re-weighted Signal-to-noise')
  pylab.show()

challenge_matched_filter(files[0], 32)